In [5]:
import subprocess
import pkg_resources
from concurrent.futures import ThreadPoolExecutor

In [6]:
failed_packages = []
path = 'requirements.txt'

def get_installed_packages():
    """
        Returns a dictionary of installed Python libraries and their versions.

        Uses `pip freeze` to list installed packages and their versions.

        Returns:
            dict: A dictionary with package names as keys and versions as values.
    """
    installed = {}
    try:
        output = subprocess.run(
            # stdout=subprocess.PIPE captures standard output
            # stdout=subprocess.PIPE captures standard error 
            ['pip', 'freeze'], stdout=subprocess.PIPE, stderr=subprocess.PIPE
        )
        lines = output.stdout.decode().splitlines()   # decodes the captured standard output from the bytes to a string into a list of line
        for line in lines:
            if '==' in line:
                name, version = line.split('==')
                installed[name.lower()] = version   # package : version -> in dict
    except Exception as e:
        print(f"Error fetching installed package {e}")
    return installed

In [7]:
def needs_update(package, installed_packages):
    """
    Checks if a package needs to be installed or updated.

    Args:
        package (str): The package to check, optionally with a specific version.
        installed_packages (dict): A dictionary of installed packages with their versions.

    Returns:
        bool: True if the package needs to be installed or updated, False otherwise.
    """
    package_name = package.split('==')[0].strip().lower()
    if package_name not in installed_packages:
        return True
    if '==' in package:
        required_version = package.split('==')[1].strip()
        return installed_packages[package_name] != required_version  # if the current version does not match the latest version then return true 
    return False  # package in installed but no specific version is required

In [8]:
def install_package(package):
    """
    Installs or upgrades a package to the latest version.

    Args:
        package (str): The package to install or upgrade.

    Returns:
        bool: True if the installation or upgrade was successful, False otherwise.
    """
    try:
        subprocess.run(
            ['pip', 'install', '-U', package],
            check=True,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE
        )
        print(f'Successfully installed or upgraded {package}')   # the above line has -U so automatic updates
        return True
    except subprocess.CalledProcessError as e:  # this line catches errors specifically related to the subprocess.run call failing (eg if pip install command fails) assigning the exception to the variable e
        print(f'Failed to install {package} : {e.stderr.decode()}')
        failed_packages.append(package)
        return False

In [9]:
def main():
    installed_packages = get_installed_packages()                     # getting all the packages that have already been installed 
    print('---')
    print(f'Installed packages : {len(installed_packages)}')
    
    try:
        with open(path, 'r') as file:
            package = [line.strip() for line in file if line.strip()] # making sure that only package names are only in the list emply lines are also taken care of 
    except FileNotFoundError:
        print(f"Error: {path} not found")
        package = []
                
    all_packages = set(package + list(installed_packages.keys()))     # All the package we want to install and the current packages in the local system will be upto date with the lates version
        
    print(f"All packages to process: {sorted(all_packages)}")
        
    with ThreadPoolExecutor(max_workers=4) as executor:
        results = list(executor.map(install_package, all_packages))
        
    if failed_packages:
        print('\nThe following package failed to install')
        for pkg in failed_packages:
            print(f'- {pkg}')
    else:
        print('\nAll packages installed successfully')

In [10]:
main()

---
Installed packages : 385
Error: requirements.txt not found
All packages to process: ['aiohappyeyeballs', 'aiohttp', 'aiosignal', 'aiosqlite', 'alembic', 'altair', 'annotated-types', 'ansi2html', 'anthropic', 'anyio', 'apispec', 'apscheduler', 'argcomplete', 'argon2-cffi', 'argon2-cffi-bindings', 'arrow', 'asgiref', 'asttokens', 'async-lru', 'asyncer', 'attrs', 'babel', 'backoff', 'bcrypt', 'beautifulsoup4', 'bleach', 'blinker', 'blis', 'bokeh', 'bokeh_sampledata', 'bs4', 'build', 'cachelib', 'cachetools', 'catalogue', 'certifi', 'cffi', 'charset-normalizer', 'chroma-hnswlib', 'chromadb', 'click', 'clickclick', 'cloudpathlib', 'cloudpickle', 'colorama', 'colorcet', 'coloredlogs', 'colorlog', 'comm', 'confection', 'configupdater', 'connexion', 'contourpy', 'cron-descriptor', 'croniter', 'cryptography', 'cycler', 'cymem', 'dash', 'dash-core-components', 'dash-html-components', 'dash-table', 'dataclasses-json', 'datasets', 'debugpy', 'decorator', 'defusedxml', 'deprecated', 'dill', 'di